In [ ]:
# 导入必要的包
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# 导入必要的包
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score


from sklearn import datasets


from sklearn.preprocessing import LabelEncoder

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_curve, auc
from sklearn import preprocessing
from sklearn import neighbors

import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option('display.notebook_repr_html', False)

%matplotlib inline
plt.style.use('seaborn-white')

In [ ]:
#导入数据
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')


In [ ]:
# 检查train data中的数据类型
app_train.dtypes.value_counts()

In [ ]:
# 编码种类不超过2的分类变量
le = LabelEncoder()
le_count = 0
 
# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
#处理性别变量
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)
 
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
targ = app_train['TARGET'] #save the column "TARGET"
 
# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
 
# Add the target back to the train data
app_train['TARGET'] = targ
 
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
# Even though we found some null data in the train data, we can still use correlations to select appropriate variables
# Find correlations beween variables with the target 
# and rank them in order to pick up variables relatively highly related to the response

correlations = app_train.corr()['TARGET'].sort_values()
 
# print those correlation coefficient
print('Variables with TOP 15 Positive Correlation coefficient:\n', correlations.tail(15))
print('\nVariables with TOP 15 Negative Correlation coefficient:\n', correlations.head(15))

In [ ]:
# Make a new dataframe for variables selected before, i.e. the polynomial features
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH','TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
 
# To avoid those null data affect the regression model, we need to replace them with appropriate values    
# We adopt the function ---- imputer to do that
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
 
# Similarly, we leave the column "TARGET" unaffected    
poly_target = poly_features['TARGET']
poly_features_new = poly_features.drop(columns = ['TARGET'])
 
# impute missing values in both train and test data
poly_features_new = imputer.fit_transform(poly_features_new)
poly_features_test_new = imputer.transform(poly_features_test)
 
from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
# Calculate the significance of poly features and check whether each of them should be in the regression
Version_1 = sm.add_constant(poly_features_new)
est = sm.Logit(poly_target , Version_1).fit()
est.summary().tables[1]

In [ ]:
# Now that all of variables mentioned above are appropriate, we try to build a regression model
regr = skl_lm.LogisticRegression()
regr.fit(poly_features_new, poly_target)

In [ ]:
pred = regr.predict(poly_features_new)
cm_log = confusion_matrix(poly_target, pred)
cm_df = pd.DataFrame(cm_log.T, index=regr.classes_,
                     columns=regr.classes_)
cm_df.index.name = 'Predicted'
cm_df.columns.name = 'True'
print(cm_df)

In [ ]:
pred_p = regr.predict_proba(poly_features_new)
cm_df = pd.DataFrame({'True': poly_target, 'Pred': pred_p[:,1] > .2})
cm_df.Pred.replace(to_replace={True:'Yes', False:'No'}, inplace=True)
print(cm_df.groupby(['True', 'Pred']).size().unstack('True').T)
print(classification_report(poly_target, pred_p[:,1] > .2))

In [ ]:
# Train the polynomial features
poly_transformer.fit(poly_features_new)
 
# Transform the features
poly_features_trans = poly_transformer.transform(poly_features_new)
poly_features_test_trans = poly_transformer.transform(poly_features_test_new)
print('Polynomial Features shape: ', poly_features_trans.shape)
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

In [ ]:
# Create a dataframe of the features 
poly_features_DF = pd.DataFrame(poly_features_trans, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

poly_features_test_DF = pd.DataFrame(poly_features_test_trans, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))
 
# Add the target into the data
poly_features_DF['TARGET'] = poly_target
 
# Find the correlations with the target and print them
correlations_2 = poly_features_DF.corr()['TARGET'].sort_values()
 
print(correlations_2.head(15))
print(correlations_2.tail(15))


In [ ]:
# Only choose variables with a correlation coefficient more than 0.150000
try_set = correlations_2.head(10)._stat_axis.values.tolist() + correlations_2.tail(5)._stat_axis.values.tolist()
try_set

In [ ]:
try_set2=try_set[:-2]
try_set2

In [ ]:
#calculate the significance of candidates of new features
Version_2 = sm.add_constant(poly_features_DF[try_set2])
est2 = sm.Logit(poly_target , Version_2).fit()
est2.summary().tables[1]

In [ ]:
#挑选出有用的参数并再计算验证
Version_3=poly_features_DF[['EXT_SOURCE_2 EXT_SOURCE_3','EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3',
                               'EXT_SOURCE_2 EXT_SOURCE_3^2','EXT_SOURCE_1 EXT_SOURCE_3',
                               'EXT_SOURCE_2','EXT_SOURCE_3','EXT_SOURCE_1 EXT_SOURCE_2 DAYS_BIRTH',
                               'EXT_SOURCE_2 DAYS_BIRTH','EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH']]
est3 = sm.Logit(poly_target , Version_3).fit()
est3.summary().tables[1]

In [ ]:
regr2 = skl_lm.LogisticRegression()
regr2.fit(Version_3, poly_target)

pred2 = regr2.predict(Version_3)
cm_log2 = confusion_matrix(poly_target, pred2)
cm_df2 = pd.DataFrame(cm_log2.T, index=regr2.classes_,
                     columns=regr2.classes_)
cm_df2.index.name = 'Predicted'
cm_df2.columns.name = 'True'
print(cm_df2)

In [ ]:
# set 0.2 as a standard to classify the prediction
pred_p2 = regr2.predict_proba(Version_3)
cm_df2 = pd.DataFrame({'True': poly_target, 'Pred': pred_p2[:,1] > .2})
cm_df2.Pred.replace(to_replace={True:'Yes', False:'No'}, inplace=True)
print(cm_df2.groupby(['True', 'Pred']).size().unstack('True').T)
print(classification_report(poly_target, pred_p2[:,1] > .2))

In [ ]:
# ROC curve of Logistic Regression1
false_pos_rate1, true_pos_rate1, _ = roc_curve(poly_target, pred_p[:,1])
roc_auc1 = auc(false_pos_rate1, true_pos_rate1)

# ROC curve of Logistic Regression2
false_pos_rate2, true_pos_rate2, _ = roc_curve(poly_target, pred_p2[:,1])
roc_auc2 = auc(false_pos_rate2, true_pos_rate2)


fig, (ax1) = plt.subplots(1,  figsize=(10,10))
ax1.plot(false_pos_rate1, true_pos_rate1, label='Logistic Regression1(area = %0.2f)' % roc_auc1, color='b')
ax1.plot(false_pos_rate2, true_pos_rate2, label='Logistic Regression2(area = %0.2f)' % roc_auc2, color='r')
ax1.set_title('Dataset')



for ax in fig.axes:
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([-0.05, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc="lower right")

In [ ]:
# obtain the final result
pred3 =regr2.predict_proba(poly_features_test_DF[['EXT_SOURCE_2 EXT_SOURCE_3',
                                         'EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3',
                                         'EXT_SOURCE_2 EXT_SOURCE_3^2',
                                         'EXT_SOURCE_1 EXT_SOURCE_3',
                                         'EXT_SOURCE_2','EXT_SOURCE_3',
                                         'EXT_SOURCE_1 EXT_SOURCE_2 DAYS_BIRTH',
                                         'EXT_SOURCE_2 DAYS_BIRTH'
                                         ,'EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH']])


In [ ]:

result = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': pred3[:,1]})

result

In [ ]:
# save the data for submission
result.to_csv('result.csv', index=False)